In [5]:
pip install pyqt5

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pyqt5-tools

Note: you may need to restart the kernel to use updated packages.


In [1]:
!pyuic5 -x "MING PHOTO 임시저장.ui" -o MING_PHOTO.py

In [ ]:
import cv2
import numpy as np
import sys
import time
from PyQt5 import QtGui, QtCore
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QLabel, QFileDialog, QColorDialog,  QInputDialog
from PyQt5.QtCore import Qt, QTimer
from PyQt5.QtGui import QImage, QPixmap
from MING_PHOTO import Ui_MainWindow

class Window(QMainWindow):
    def __init__(self, ui_file_path):
        super(Window, self).__init__()
        # 기본 UI 로드
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self) 
        # 시작 시에 첫 번째 페이지 표시
        self.ui.stackedWidget_.setCurrentIndex(0)
        # page_start 버튼 함수 연결
        self.ui.pushButton.clicked.connect(self.goCamera) #사진촬영창 이동
        self.ui.pushButton_2.clicked.connect(self.goGallery) #사진 선택창 이동
        self.ui.pushButton_4.clicked.connect(self.close_window) #창 종료
        #page_camera 버튼 함수 연결
        self.ui.horizontalSlider.valueChanged.connect(self.update_frame)#zoom 기능
        self.ui.pushButton_10.clicked.connect(self.mirrorPlus)#좌우반전 기능
        self.ui.pushButton_5.clicked.connect(self.click)#찰칵 기능 -> 저장까지
        self.ui.pushButton_6.clicked.connect(self.goMain)#첫화면 돌아가기
        #page_cameraSave 
        self.ui.pushButton_8.clicked.connect(self.goEdit) #꾸미기 창 이동 함수
        self.ui.pushButton_7.clicked.connect(self.goCamera)#page2로 이동
        #page_gallery
        self.ui.pushButton_9.clicked.connect(self.goMain)#첫화면 돌아가기
        self.ui.pushButton_3.clicked.connect(self.goGallery) #사진 선택 함수
        self.ui.pushButton_11.clicked.connect(self.goEdit) #꾸미기 창 이동 함수
        #page_edit
        self.ui.horizontalSlider_3.valueChanged.connect(self.brightness)
        self.ui.horizontalSlider_2.valueChanged.connect(self.contrast)
        #self.ui.pushButton_16.clicked.connect(self.apply_changes)
        self.ui.pushButton_15.clicked.connect(self.save_image) #저장
        self.ui.label_15.mousePressEvent = lambda event:self.select()
        self.ui.pushButton_14.clicked.connect(self.refresh)
        self.ui.pushButton_13.clicked.connect(self.goMain)
        self.ui.pushButton_12.clicked.connect(self.close_window)
        self.ui.label_5.mousePressEvent = lambda event: self.on_label_5_click(event)
        #self.ui.label_11.mousePressEvent = lambda event: self.flat()
        self.ui.label_9.mousePressEvent = lambda event: self.crop() #자르기
        self.ui.label_13.mousePressEvent = lambda event: self.on_label_13_click()
        self.ui.label_14.mousePressEvent = lambda event: self.pick_color()  # 레이블 클릭 시 색상 선택
        self.ui.spinBox.valueChanged.connect(self.update_thickness)
        self.ui.label_16.mousePressEvent = lambda event: self.mosaic()
        self.ui.label_10.mousePressEvent = lambda event: self.flip() #좌우반전
        self.ui.label_12.mousePressEvent = lambda event: self.rotate( )#회전
        self.ui.label_19.mousePressEvent = lambda event: self.gray( )#흑백필터
        self.ui.label_20.mousePressEvent = lambda event: self.blur( )#뽀얀필터
        self.ui.label_21.mousePressEvent = lambda event: self.sharp( )#선명한필터
        self.ui.label_43.mousePressEvent = lambda event: self.snow_effect( )#snow효과
        self.ui.label_30.mousePressEvent = lambda event: self.on_label_30_click()
        self.ui.label_37.mousePressEvent = lambda event: self.on_label_37_click()
        self.ui.label_38.mousePressEvent = lambda event: self.on_label_38_click()
        self.ui.label_8.mousePressEvent = lambda event:self.collage_ver1()
        # mirror 변수 선언
        self.mirror = 0  
        self.filePath = None
        self.roi = None
        # 임시로 저장할 이미지 변수
        self.temp_image = None
        self.original_image = None
        #새로고침할 때 불러올 이미지 저장
        self.refresh_image = None
        #카메라 촬영시, 화면에 보이는 이미지를 캡처하기 위해, 화면에 마지막으로 표시 중인 이미지를 저장하는 변수 
        self.last_displayed_frame = None
        self.color = (0, 0, 0) #색상 저장 변수, 기본값_검정
        self.thickness = 1.0 #두께 저장 변수, 기본값 1
        self.tree_sticker_mode = False
        self.santa_sticker_mode = False
        self.snow_sticker_mode = False
        self.drawing = False
        self.pt1, pt2 = (-1, -1), (-1, -1)
        
    #창 전환, 종료 함수들
    def goCamera(self):
        self.ui.stackedWidget_.setCurrentIndex(1)
        self.enable_camera() #카메라를 page2에서만 작동하도록 page2에 들어갈 때 카메라를 작동시켜줌

    def goGallery(self):
        self.ui.stackedWidget_.setCurrentIndex(3)
        filePath, _ = QFileDialog.getOpenFileName(self, "사진 선택", './', "이미지 파일 (*.png *.jpg *.bmp *.jpeg);;모든 파일 (*)")

        with open(filePath, 'rb') as f:
            # 이미지를 uint8 ndarray로 열고 opencv로 decode
            img_data = f.read()
            nparr = np.frombuffer(img_data, np.uint8)
            self.temp_image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        self.update_image_to_label(self.ui.label_6)
        
    def select(self):
        filePath, _ = QFileDialog.getOpenFileName(self, "사진 선택", './', "이미지 파일 (*.png *.jpg *.bmp *.jpeg);;모든 파일 (*)")

        with open(filePath, 'rb') as f:
            # 이미지를 uint8 ndarray로 열고 opencv로 decode
            img_data = f.read()
            nparr = np.frombuffer(img_data, np.uint8)
            self.temp_image = cv2.imdecode(nparr, cv2.IMREAD_COLOR) 
        self.refresh_image = self.temp_image.copy()
        self.update_image_to_label(self.ui.label_5)
    
    def close_window(self):
         self.close()
        
    def goMain(self):
        current_index = self.ui.stackedWidget_.currentIndex()
        if current_index == 1:  # stackedWidget의 현재 인덱스가 1일 때만 실행
            self.disable_camera()  # 페이지2를 나가면서 카메라 종료
        self.ui.stackedWidget_.setCurrentIndex(0)

    def goEdit(self):
        self.ui.stackedWidget_.setCurrentIndex(4)
        self.refresh_image = self.temp_image.copy()
        self.update_image_to_label(self.ui.label_5)
        label = QLabel()
        icon_paths = [
             "tree_icon.png","santa_icon.png","snow_icon.png"   
        ]
        label_list = [
            self.ui.label_30, self.ui.label_37, self.ui.label_38
        ]
        for i, icon_path in enumerate(icon_paths):
            image_path = rf"icon\{icon_path}"
            pixmap = QPixmap(image_path)
            label = label_list[i]
            label.setPixmap(pixmap) 
            
    # 카메라 활성화 
    def enable_camera(self):
        self.cap = cv2.VideoCapture(0)  # 0번 카메라 연결
        self.timer = QTimer(self) #타이머 생성
        self.timer.timeout.connect(self.update_frame)#타이머 시간 종료마다 함수연결 -> 화면 실시간 출력
        self.timer.start(30) #타이머 시간 간격 설정
        self.ui.horizontalSlider.setValue(0)
        self.camera_enabled = True
        
    #카메라 화면 띄워주는 함수
    def update_frame(self):
        ret, frame = self.cap.read()
        # 좌우반전 여부 결정 (변수 홀/짝)
        if self.mirror % 2 == 1:
            frame = cv2.flip(frame, 1)
        # 프레임을 잘 받은 경우
        if ret:
            zoom_factor = 1 - (self.ui.horizontalSlider.value() / 100.0)
            cropped_frame = self.crop_and_resize(frame, zoom_factor)
            self.last_displayed_frame = cropped_frame
            q_image = self.convert_frame_to_qimage(cropped_frame)
            self.ui.label_3.setPixmap(QPixmap.fromImage(q_image))
            
    def crop_and_resize(self, frame, zoom_factor):
        h, w, ch = frame.shape
        center_y, center_x = h // 2, w // 2
        crop_size = int(min(h, w) * max(0, zoom_factor) / 2)
        start_y = max(0, center_y - crop_size)
        end_y = min(h, center_y + crop_size)
        start_x = max(0, center_x - crop_size)
        end_x = min(w, center_x + crop_size)
        
        cropped_frame = frame[start_y:end_y, start_x:end_x]
        return cv2.resize(cropped_frame, (w, h))

    #사용자가 좌우 반전 버튼을 누를 떄 마다 1씩 증가
    def mirrorPlus(self):
        self.mirror += 1
        # 미러가 홀수일 경우, 바로 플립을 적용합니다.
        if self.mirror % 2 == 1:
            self.flip()
        
    # 카메라 비활성화 
    def disable_camera(self):
        if hasattr(self, 'cap') and self.cap.isOpened():
            self.cap.release()
        self.camera_enabled = False

    def click(self):
        self.temp_image = self.last_displayed_frame
        self.update_image_to_label(self.ui.label_4)
        # 꾸미기 선택할 지 말지 선택하는 창 이동
        self.ui.stackedWidget_.setCurrentIndex(2)
        
    def refresh(self):
        self.temp_image = self.refresh_image.copy()  
        self.update_image_to_label(self.ui.label_5)  
        self.ui.horizontalSlider_2.setValue(0)
        self.ui.horizontalSlider_3.setValue(0)
    
    def crop(self): 
        self.temp_image_bgr = cv2.cvtColor(self.temp_image, cv2.COLOR_RGB2BGR)
        x, y, w, h = cv2.selectROI(self.temp_image, False)
        
        if w and h:
            self.roi = self.temp_image[y:y+h, x:x+w]
            self.temp_image = self.roi
            self.update_image_to_label(self.ui.label_5)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

    def flip(self):
        self.temp_image = cv2.flip(self.temp_image, 1)  # 1은 좌우반전
        self.update_image_to_label(self.ui.label_5)

    def rotate(self):
        self.temp_image = cv2.rotate(self.temp_image, cv2.ROTATE_90_CLOCKWISE)  # 시계방향으로 90도 회전
        self.update_image_to_label(self.ui.label_5)

    def on_label_13_click(self):
        self.drawing = False  # 초기화
        self.pt1, self.pt2 = (-1, -1), (-1, -1)  # 초기화

        cv2.namedWindow('draw')
        cv2.setMouseCallback('draw', self.draw)

        while True:
            cv2.imshow('draw', self.temp_image)
            key = cv2.waitKey(1) & 0xFF

            if key == 27:  #esc
                cv2.destroyAllWindows()
                break
            elif key == 13:  # 엔터 키를 누르면 창 닫기
                cv2.destroyAllWindows()
                break
            self.update_image_to_label(self.ui.label_5)  # 그림을 갱신하고 label에 반영하기
        cv2.destroyAllWindows()

    def draw(self, event, x, y, flags, param):
        global drawing, pt1, pt2

        if event == cv2.EVENT_LBUTTONDOWN:
            self.drawing = True
            self.pt1 = (x, y)

        elif event == cv2.EVENT_LBUTTONUP:
            self.drawing = False

        elif event == cv2.EVENT_MOUSEMOVE:
            if self.drawing:
                self.pt2 = (x, y)
                self.update_thickness
                #원과 선을 함께 그려서 글리터선 같은 효과
                cv2.line(self.temp_image, self.pt1, self.pt2, self.color, int(self.thickness))
                cv2.circle(self.temp_image, self.pt2, int(self.thickness)+1, (255, 255, 255), -1)
                self.pt1 = self.pt2
                
    def pick_color(self):
        color = QColorDialog.getColor()
        bgr_color = color.getRgb()[2], color.getRgb()[1], color.getRgb()[0]
        self.color = bgr_color
            
    def update_thickness(self, value):
        self.thickness = value
    
    #흑백_필터
    def gray(self):
        self.temp_image = cv2.cvtColor(self.temp_image, cv2.COLOR_BGR2GRAY)
        self.temp_image = cv2.cvtColor(self.temp_image, cv2.COLOR_GRAY2RGB) 
        self.update_image_to_label(self.ui.label_5)

    #뽀얗게_필터    
    def blur(self):
        self.temp_image = cv2.blur(self.temp_image, (3, 3))
        self.update_image_to_label(self.ui.label_5)
    
    #선명하게_필터
    def sharp(self):
        mask = np.asarray([[-1, -1, -1],
                   [-1,  9, -1],
                   [-1, -1, -1]], dtype=np.float32)
        self.temp_image = cv2.filter2D(self.temp_image,-1, mask)
        self.update_image_to_label(self.ui.label_5)
        
    def snow_effect(self):
        # 스티커 이미지의 크기를 가져옴
        sticker = cv2.imread("icon/snow_effect.png", cv2.IMREAD_UNCHANGED)
        sticker_h, sticker_w, _ = sticker.shape

        # 스티커를 적용할 좌표 계산 (좌상단에서부터 시작)
        x_sticker_center = 0
        y_sticker_center = 0

        self.snow_effect_apply(x_sticker_center, y_sticker_center)
        self.update_image_to_label(self.ui.label_5)

    def snow_effect_apply(self, x, y):
        # 이미지 처리 작업 수행
        self.temp_image = self.apply_sticker(self.temp_image, "icon/snow_effect.png", x, y)
        
    def brightness(self):
        self.update_display()

    def contrast(self):
        self.update_display()

    def update_display(self):
        if self.original_image is not None:
            alpha = (self.ui.horizontalSlider_2.value() + 100.0) / 100.0 #대비 조절값
            bright = self.ui.horizontalSlider_3.value()  # 밝기 조절값
            beta = bright - 128 * (alpha - 1.0)
            self.temp_image = cv2.convertScaleAbs(self.original_image, alpha=alpha, beta=beta)
        else:
            self.original_image = self.temp_image.copy()
        self.update_image_to_label(self.ui.label_5)

    def on_label_30_click(self):
        self.tree_sticker_mode = True
        
    def on_label_37_click(self):
        self.santa_sticker_mode = True
        
    def on_label_38_click(self):
        self.snow_sticker_mode = True

    def on_label_5_click(self, event):
        x = event.x()
        y = event.y()

        # PyQt 좌표를 OpenCV 좌표로 변환
        x_opencv = int(x * self.temp_image.shape[1] / self.ui.label_5.width())
        y_opencv = int(y * self.temp_image.shape[0] / self.ui.label_5.height())

        if self.tree_sticker_mode:
            # 스티커 이미지의 크기를 가져옴
            sticker = cv2.imread("icon/tree.png", cv2.IMREAD_UNCHANGED)
            sticker_h, sticker_w, _ = sticker.shape

            x_sticker_center = x_opencv - sticker_w // 2
            y_sticker_center = y_opencv - sticker_h // 2

            self.tree_sticker(x_sticker_center, y_sticker_center)
            self.tree_sticker_mode = False
            self.update_image_to_label(self.ui.label_5)
            
        elif self.santa_sticker_mode:
            sticker = cv2.imread("icon/santa.png", cv2.IMREAD_UNCHANGED)
            sticker_h, sticker_w, _ = sticker.shape
            
            x_sticker_center = x_opencv - sticker_w // 2
            y_sticker_center = y_opencv - sticker_h // 2

            self.santa_sticker(x_sticker_center, y_sticker_center)
            self.santa_sticker_mode = False
            self.update_image_to_label(self.ui.label_5)
        
        elif self.snow_sticker_mode:
            sticker = cv2.imread("icon/snow.png", cv2.IMREAD_UNCHANGED)
            sticker_h, sticker_w, _ = sticker.shape
            
            x_sticker_center = x_opencv - sticker_w // 2
            y_sticker_center = y_opencv - sticker_h // 2

            self.snow_sticker(x_sticker_center, y_sticker_center)
            self.snow_sticker_mode = False
            self.update_image_to_label(self.ui.label_5)

    def tree_sticker(self, x, y):
        # 이미지 처리 작업 수행
        self.temp_image = self.apply_sticker(self.temp_image, "icon/tree.png", x, y)

    def santa_sticker(self, x, y):
        self.temp_image = self.apply_sticker(self.temp_image, "icon/santa.png", x, y)
    
    def snow_sticker(self, x, y):
        self.temp_image = self.apply_sticker(self.temp_image, "icon/snow.png", x, y)
        
    # OpenCV에서의 스티커 적용
    def apply_sticker(self, image, sticker_path, x, y):
        sticker = cv2.imread(sticker_path, cv2.IMREAD_UNCHANGED)
        if sticker is None:
            print(f"스티커 파일을 읽을 수 없습니다: {sticker_path}")
            return image

        h, w, _ = sticker.shape[:3]

        # 이미지 경계 안
        x_start = max(0, x)
        x_end = min(image.shape[1], x + w)
        y_start = max(0, y)
        y_end = min(image.shape[0], y + h)

        # 이미지 경계를 벗어난 경우
        sticker_x_start = max(0, -x)
        sticker_x_end = min(w, w - (x + w - image.shape[1]))
        sticker_y_start = max(0, -y)
        sticker_y_end = min(h, h - (y + h - image.shape[0]))
        
        roi = image[y_start:y_end, x_start:x_end]

        # mask 생성
        sticker_alpha = sticker[sticker_y_start:sticker_y_end, sticker_x_start:sticker_x_end, 3] / 255.0
        sticker_alpha = cv2.resize(sticker_alpha, (x_end - x_start, y_end - y_start))  
        sticker_alpha = sticker_alpha[:, :, np.newaxis] 
        
        mask = np.zeros_like(roi)
        mask[:, :] = sticker_alpha
        roi_resized = cv2.resize(roi, (x_end - x_start, y_end - y_start))  
        image[y_start:y_end, x_start:x_end] = (1.0 - mask) * roi_resized + mask * sticker[sticker_y_start:sticker_y_end, sticker_x_start:sticker_x_end, :3]

        return image

    def mosaic(self):
        self.kernel_size = 30
        self.temp_image_bgr = cv2.cvtColor(self.temp_image, cv2.COLOR_RGB2BGR)

        x, y, w, h = cv2.selectROI(self.temp_image, False)

        if w>0 and h>0:
            self.roi = self.temp_image[y:y+h, x:x+w]
            self.roi = cv2.blur(self.roi, (self.kernel_size, self.kernel_size))
            self.temp_image[y:y+h, x:x+w] = self.roi
            self.update_image_to_label(self.ui.label_5)      
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            
    def collage_ver1(self):
        self.original_image=None
        self.temp_image = None
        self.update_image_to_label(self.ui.label_5)
        image_paths, _ = QFileDialog.getOpenFileNames(self, "사진 선택", './', "이미지 파일 (*.png *.jpg *.bmp *.jpeg);;모든 파일 (*)")

        images = []
        for i, image_path in enumerate(image_paths):
            with open(image_path, 'rb') as f:
                img_data = f.read()
                nparr = np.frombuffer(img_data, np.uint8)
                image_var_name = f"image_{i + 1}"
                setattr(self, image_var_name, cv2.imdecode(nparr, cv2.IMREAD_COLOR))
                images.append(getattr(self, image_var_name))

        collage_image = self.create_collage(images)
        self.temp_image = collage_image

        sticker_path = "icon/ver1.png"
        x_sticker_center = 0
        y_sticker_center = 0
        self.temp_image = self.apply_sticker(self.temp_image, sticker_path, x_sticker_center, y_sticker_center)
        self.update_image_to_label(self.ui.label_5)


    def create_collage(self, images):
        # 목표 크기 설정
        target_width, target_height = 640, 360

        collage_image = np.zeros((target_height * 2, target_width * 2, 3), dtype=np.uint8)

        for i, image in enumerate(images):
            aspect_ratio = image.shape[1] / image.shape[0]

            if target_width / aspect_ratio <= target_height:
                new_width = int(target_width)
                new_height = int(target_width / aspect_ratio)
            else:
                new_width = int(target_height * aspect_ratio)
                new_height = int(target_height)

            # 이미지를 새로운 크기로 조절
            resized_image = cv2.resize(image, (target_width, target_height), interpolation=cv2.INTER_AREA)

            if len(resized_image.shape) == 2:
                resized_image = cv2.cvtColor(resized_image, cv2.COLOR_GRAY2BGR)
            row = i // 2
            col = i % 2
            collage_image[row * target_height:(row + 1) * target_height, col * target_width:(col + 1) * target_width, :] = resized_image
        return collage_image
        
    def save_image(self):
        if self.temp_image is not None:
            timestamp = int(time.time())
            new_path = f"save_image/capture_{timestamp}.png"  
            cv2.imwrite(new_path, self.temp_image)

    def convert_frame_to_qimage(self, frame):
        rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w, ch = rgb_image.shape
        bytes_per_line = ch * w
        q_image = QImage(rgb_image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        return q_image
    
    def update_image_to_label(self, label):
        if self.temp_image is not None:
            pixmap = QtGui.QPixmap.fromImage(self.convert_frame_to_qimage(self.temp_image))
            pixmap = pixmap.scaled(label.size(), QtCore.Qt.KeepAspectRatio)
            label.setPixmap(pixmap)

if __name__ == "__main__":
    app = QApplication(sys.argv)

    window = Window("path/to/MING_PHOTO.ui")
    window.show()

    sys.exit(app.exec_())